In [1]:
import pandas as pd
import censusgeocode as cg
import re
import time
import geopandas 
import shapely.wkt
from shapely.geometry import Point, Polygon
import time
import datetime

In [2]:
evictions=pd.read_csv('/Users/renasong/Documents/2020su/proj/Eviction_Notices.csv')

/Users/renasong/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
evictions=evictions[['Address','Eviction Notice Source Zipcode','File Date','Neighborhoods - Analysis Boundaries','Location','Non Payment',
                    'Breach','Nuisance','Illegal Use','Failure to Sign Renewal','Access Denial','Unapproved Subtenant',
                     'Owner Move In','Demolition','Capital Improvement','Substantial Rehab','Ellis Act WithDrawal',
                     'Condo Conversion','Roommate Same Unit','Other Cause','Late Payments','Lead Remediation','Development',
                     'Good Samaritan Ends']]

evictions=evictions.rename(columns={'Neighborhoods - Analysis Boundaries':'Neighborhood','Eviction Notice Source Zipcode':'Zipcode'})


In [4]:
evictions=evictions.fillna('')

In [5]:
locations=evictions['Location'].to_numpy()

In [6]:
cd=[re.findall("(\-\\d+\.\\d+|\\d+\.\\d+)",loc) for loc in locations]

In [7]:
points=[]
for c in cd:
    if len(c)==2:
        points.append(Point(float(c[0]),float(c[1])))
    else:
        points.append('')

In [12]:
cd

[['-122.40960340577871', '37.75282207491775'],
 ['-122.43079515860666', '37.77349689420376'],
 ['-122.39899123962185', '37.75857987579773'],
 ['-122.48045074948287', '37.71901186772143'],
 ['-122.48225520781153', '37.7436977235424'],
 ['-122.40169229274082', '37.7865888594111'],
 ['-122.41901915701537', '37.78264403543776'],
 ['-122.43830450573293', '37.7901326817108'],
 ['-122.40169229274082', '37.7865888594111'],
 ['-122.42634620019078', '37.77268385160834'],
 ['-122.44175732162306', '37.756715217973'],
 ['-122.44898765370768', '37.78152103879187'],
 ['-122.40774688106724', '37.79362789496296'],
 ['-122.49698158269595', '37.741174328695585'],
 ['-122.41388136992207', '37.7131682413959'],
 ['-122.42423621296547', '37.787289037334'],
 [],
 ['-122.42218826152258', '37.77269482407448'],
 ['-122.46092794537107', '37.78020044426889'],
 ['-122.50807101147625', '37.75939478913729'],
 ['-122.40169229274082', '37.7865888594111'],
 ['-122.50191616518941', '37.75691304662955'],
 ['-122.473917054

In [8]:
sf=pd.read_csv('sf polygons.csv')

In [9]:
polys = [shapely.wkt.loads(st) for st in sf['geometry']]

In [10]:
res=[]
for point in points:
    if type(point)!=shapely.geometry.point.Point:
        res.append(0)
    else:
        count=0
        for poly in polys:
            if poly.contains(point):
                res.append(poly)
                count+=1
        if count<1:
            res.append(0)
                

KeyboardInterrupt: 

In [170]:
pointsdf = pd.DataFrame(points)
pointsdf['polygons'] = res

pointsdf['poly']=[str(pt) for pt in pointsdf['polygons']]
sf['poly']=[str(pt) for pt in sf['geometry']]
df=pd.merge(pointsdf,sf,left_on='poly',right_on='poly')

In [171]:
df['point']=df[0]
df=df[['point','geometry','TRACTCE']]

In [172]:
df.columns

Index(['point', 'geometry', 'TRACTCE'], dtype='object')

In [173]:
evictions.columns

Index(['Address', 'Zipcode', 'File Date', 'Neighborhood', 'Location',
       'Non Payment', 'Breach', 'Nuisance', 'Illegal Use',
       'Failure to Sign Renewal', 'Access Denial', 'Unapproved Subtenant',
       'Owner Move In', 'Demolition', 'Capital Improvement',
       'Substantial Rehab', 'Ellis Act WithDrawal', 'Condo Conversion',
       'Roommate Same Unit', 'Other Cause', 'Late Payments',
       'Lead Remediation', 'Development', 'Good Samaritan Ends'],
      dtype='object')

In [174]:
df['pt_h']=[str(pt) for pt in df['point']]


In [225]:

df1=df[['point','pt_h','TRACTCE']].merge(evictions[['Location','File Date','Non Payment', 'Breach', 'Nuisance', 
        'Illegal Use','Failure to Sign Renewal', 'Access Denial', 'Unapproved Subtenant',
       'Owner Move In', 'Demolition', 'Capital Improvement',
       'Substantial Rehab', 'Ellis Act WithDrawal', 'Condo Conversion',
       'Roommate Same Unit', 'Other Cause', 'Late Payments',
       'Lead Remediation', 'Development', 'Good Samaritan Ends']],left_on='pt_h',right_on='Location')
coords = df1.point.apply(lambda x: x.coords.xy)



In [226]:
df1['long']=[c[0][0] for c in coords]
df1['lat']=[c[1][0] for c in coords]
df1=df1.drop(['point','pt_h','Location'],axis=1)


In [227]:
df1=df1*1 


In [228]:
split=[date.rsplit('/') for date in df1['File Date']]
df1['month']=[date[0] for date in split]
df1['day']=[date[1] for date in split]
df1['year']=[int(date[2]) for date in split]
#create month, day, year cols

In [229]:
df1=df1.loc[df1['year']>2009]
#keep years 2010 and above

In [230]:
df1.columns

Index(['TRACTCE', 'File Date', 'Non Payment', 'Breach', 'Nuisance',
       'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends', 'long', 'lat', 'month', 'day', 'year'],
      dtype='object')

In [231]:
df1['Non Payment']=[int(x) for x in df1['Non Payment']]

df2=df1[['Non Payment', 'Breach', 'Nuisance',
       'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends']]
def get_reason(row):
     for c in df2.columns:
            if row[c]==1:
                return c

In [232]:
df1['reason']=df2.apply(get_reason, axis=1)
df1=df1[['TRACTCE','File Date','long','lat','month','day','year','reason']]

In [233]:
df1['date']=[datetime.datetime.fromtimestamp(time.mktime(datetime.datetime.strptime(s, "%m/%d/%Y").timetuple())) for s in df1['File Date']]
df1=df1.drop(['File Date'],axis=1)

In [234]:
df1.head()

,TRACTCE,long,lat,month,day,year,reason,date
0,22902,-122.411016,37.748408,09,01,2016,Breach,2016-09-01
1,22902,-122.411016,37.748408,03,11,2019,Nuisance,2019-03-11
2,22902,-122.411016,37.748408,09,01,2016,Breach,2016-09-01
3,22902,-122.411016,37.748408,03,11,2019,Nuisance,2019-03-11
4,22902,-122.410099,37.748427,09,27,2012,Ellis Act WithDrawal,2012-09-27


In [235]:
df1.to_csv('Evictions.csv')